In this fourth and final notebook, we will go over how to leverage the SDK to directly work interactively with a Ray cluster during development.

In [1]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [ ]:
# Create authentication object for oc user permissions
auth = TokenAuthentication(
    token = "XXXXX",
    server = "XXXXX",
    skip_tls=False
)
auth.login()

Once again, let's start by running through the same cluster setup as before:

In [3]:
# Create and configure our cluster object (and appwrapper)
cluster = Cluster(ClusterConfiguration(
    name='interactivetest',
    namespace='default',
    min_worker=2,
    max_worker=2,
    min_cpus=2,
    max_cpus=2,
    min_memory=8,
    max_memory=8,
    gpu=1,
    instascale=True,
    machine_types=["m5.xlarge", "g4dn.xlarge"]
    
))

Written to: interactivetest.yaml


In [4]:
# Bring up the cluster
cluster.up()
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster up and running!


In [5]:
cluster.details()

                      🚀 CodeFlare Cluster Details 🚀                     
                                                                          
 ╭──────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                               │ 
 │   interactivetest                                        Active ✅   │ 
 │                                                                      │ 
 │   URI: ray://interactivetest-head-svc.default.svc:10001              │ 
 │                                                                      │ 
 │   ]8;id=789787;http://ray-dashboard-interactivetest-default.apps.meyceoz-032023.psap.aws.rhperfscale.org\Dashboard🔗]8;;\                                                        │ 
 │                                                                      │ 
 │                      Cluster Resources                               │ 
 │   ╭─ Workers ──╮  ╭───────── Worker specs(each) ─────────╮           │ 
 │   │  Min  Max  │  │  Memory      CPU         GPU         │           │ 
 │   │            │  │                                      │           │ 
 │   │  2    2    │  │  8~8         2           1           │           │ 
 │   │            │  │                                      │           │ 
 │   ╰────────────╯  ╰──────────────────────────────────────╯           │ 
 ╰──────────────────────────────────────────────────────────────────────╯

RayCluster(name='interactivetest', status=<RayClusterStatus.READY: 'ready'>, min_workers=2, max_workers=2, worker_mem_min=8, worker_mem_max=8, worker_cpu=2, worker_gpu=1, namespace='default', dashboard='http://ray-dashboard-interactivetest-default.apps.meyceoz-032023.psap.aws.rhperfscale.org')

This time we will demonstrate another potential method of use: working with the Ray cluster interactively.

Using the SDK, we can get both the Ray cluster URI and dashboard URI:

In [6]:
ray_dashboard_uri = cluster.cluster_dashboard_uri()
ray_cluster_uri = cluster.cluster_uri()
print(ray_dashboard_uri)
print(ray_cluster_uri)

http://ray-dashboard-interactivetest-default.apps.meyceoz-032023.psap.aws.rhperfscale.org
ray://interactivetest-head-svc.default.svc:10001


Now we can connect directly to our Ray cluster via the Ray python client:

In [7]:
#before proceeding make sure the cluster exists and the uri is not empty
assert ray_cluster_uri, "Ray cluster needs to be started and set before proceeding"

import ray
from ray.air.config import ScalingConfig

# reset the ray context in case there's already one. 
ray.shutdown()
# establish connection to ray cluster

#install additionall libraries that will be required for model training
runtime_env = {"pip": ["transformers", "datasets", "evaluate", "pyarrow<7.0.0", "accelerate"]}

ray.init(address=f'{ray_cluster_uri}', runtime_env=runtime_env)

print("Ray cluster is up and running: ", ray.is_initialized())

Ray cluster is up and running:  True


Now that we are connected (and have passed in some package requirements), let's try writing some training code for a DistilBERT transformer model via HuggingFace (using IMDB dataset):

In [8]:
@ray.remote
def train_fn():
    from datasets import load_dataset
    import transformers
    from transformers import AutoTokenizer, TrainingArguments
    from transformers import AutoModelForSequenceClassification
    import numpy as np
    from datasets import load_metric
    import ray
    from ray import tune
    from ray.train.huggingface import HuggingFaceTrainer

    dataset = load_dataset("imdb")
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    #using a fraction of dataset but you can run with the full dataset
    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

    print(f"len of train {small_train_dataset} and test {small_eval_dataset}")

    ray_train_ds = ray.data.from_huggingface(small_train_dataset)
    ray_evaluation_ds = ray.data.from_huggingface(small_eval_dataset)

    def compute_metrics(eval_pred):
        metric = load_metric("accuracy")
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    def trainer_init_per_worker(train_dataset, eval_dataset, **config):
        model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

        training_args = TrainingArguments("/tmp/hf_imdb/test", eval_steps=1, disable_tqdm=True, 
                                          num_train_epochs=1, skip_memory_metrics=True,
                                          learning_rate=2e-5,
                                          per_device_train_batch_size=16,
                                          per_device_eval_batch_size=16,                                
                                          weight_decay=0.01,)
        return transformers.Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            compute_metrics=compute_metrics
        )

    scaling_config = ScalingConfig(num_workers=2, use_gpu=True) #num workers is the number of gpus

    # we are using the ray native HuggingFaceTrainer, but you can swap out to use non ray Huggingface Trainer. Both have the same method signature. 
    # the ray native HFTrainer has built in support for scaling to multiple GPUs
    trainer = HuggingFaceTrainer(
        trainer_init_per_worker=trainer_init_per_worker,
        scaling_config=scaling_config,
        datasets={"train": ray_train_ds, "evaluation": ray_evaluation_ds},
    )
    result = trainer.fit()

Once we want to test our code out, we can run the training function we defined above remotely on our Ray cluster:

In [9]:
#call the above cell as a remote ray function
ray.get(train_fn.remote())

(train_fn pid=293) Downloading and preparing dataset imdb/plain_text to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0...


Generating unsupervised split:  98%|█████████▊| 49182/50000 [00:07<00:00, 10168.78 examples/s]


(train_fn pid=293) Dataset imdb downloaded and prepared to /home/ray/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


Map:  98%|█████████▊| 49000/50000 [00:31<00:00, 1353.96 examples/s]


(train_fn pid=293) len of train Dataset({
(train_fn pid=293)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=293)     num_rows: 100
(train_fn pid=293) }) and test Dataset({
(train_fn pid=293)     features: ['text', 'label', 'input_ids', 'attention_mask'],
(train_fn pid=293)     num_rows: 100
(train_fn pid=293) })


...

(train_fn pid=293) 2023-04-18 15:08:08,155	WARNING util.py:244 -- The `process_trial_save` operation took 6.155 s, which may be a performance bottleneck.


(train_fn pid=293) Result for HuggingFaceTrainer_bd08a_00000:
(train_fn pid=293)   _time_this_iter_s: 276.56069135665894
(train_fn pid=293)   _timestamp: 1681855672
(train_fn pid=293)   _training_iteration: 2
(train_fn pid=293)   date: 2023-04-18_15-08-01
(train_fn pid=293)   done: true
(train_fn pid=293)   epoch: 1.0
(train_fn pid=293)   experiment_id: cb2127867b714e92b008dd8d87659908
(train_fn pid=293)   experiment_tag: '0'
(train_fn pid=293)   hostname: nteractivetest-worker-small-group-interactivetest-7m9zh
(train_fn pid=293)   iterations_since_restore: 2
(train_fn pid=293)   learning_rate: 0.0
(train_fn pid=293)   loss: 0.2252
(train_fn pid=293)   node_ip: 10.131.14.19
(train_fn pid=293)   pid: 144
(train_fn pid=293)   should_checkpoint: true
(train_fn pid=293)   step: 782
(train_fn pid=293)   time_since_restore: 745.5450358390808
(train_fn pid=293)   time_this_iter_s: 275.6667923927307
(train_fn pid=293)   time_total_s: 745.5450358390808
(train_fn pid=293)   timestamp: 1681855681

(train_fn pid=293) 2023-04-18 15:08:13,083	WARNING util.py:244 -- The `process_trial_save` operation took 1.779 s, which may be a performance bottleneck.


(train_fn pid=293) == Status ==
(train_fn pid=293) Current time: 2023-04-18 15:08:14 (running for 00:12:43.34)
(train_fn pid=293) Memory usage on this node: 3.9/15.4 GiB 
(train_fn pid=293) Using FIFO scheduling algorithm.
(train_fn pid=293) Resources requested: 0/6 CPUs, 0/2 GPUs, 0.0/22.35 GiB heap, 0.0/6.55 GiB objects
(train_fn pid=293) Result logdir: /home/ray/ray_results/HuggingFaceTrainer_2023-04-18_14-55-31
(train_fn pid=293) Number of trials: 1/1 (1 TERMINATED)
(train_fn pid=293) +--------------------------------+------------+------------------+--------+------------------+--------+-----------------+---------+
(train_fn pid=293) | Trial name                     | status     | loc              |   iter |   total time (s) |   loss |   learning_rate |   epoch |
(train_fn pid=293) |--------------------------------+------------+------------------+--------+------------------+--------+-----------------+---------|
(train_fn pid=293) | HuggingFaceTrainer_bd08a_00000 | TERMINATED | 10.13

(train_fn pid=293) 2023-04-18 15:08:14,963	INFO tune.py:777 -- Total run time: 763.54 seconds (763.34 seconds for the tuning loop).


Once complete, we can bring our Ray cluster down and clean up:

In [10]:
cluster.down()

In [ ]:
auth.logout()